  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires num

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype as is_datetime
# Carregar os dados
df = pd.read_excel('/content/drive/MyDrive/Projeto EBAC FINAL/dados_2025-02-05.xlsx')

# Configurações de exibição
pd.set_option('display.width', None)

# ------------ Exploração inicial ------------------

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2977 entries, 0 to 2976
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   cdCartaConsulta             2977 non-null   int64 
 1   nmProjeto                   2977 non-null   object
 2   sgProjeto                   2907 non-null   object
 3   nmSetor                     2891 non-null   object
 4   nmSujeito                   2977 non-null   object
 5   Regiao                      2977 non-null   object
 6   nmUnidadeFederal            2971 non-null   object
 7   nmAbrangenciaNacional       2977 non-null   object
 8   TipoFinanciamento           2977 non-null   object
 9   sgFonte                     2976 non-null   object
 10  VlDolar                     2977 non-null   object
 11  CpVlDolar                   2977 non-null   object
 12  sgMoeda                     2977 non-null   object
 13  deFase                      2977 non-null   obje

# ---------- CONVERSÃO DE DATAS ----------

In [5]:
cols_datas = [
    "dtRecebimento",
    "dtValidadeRecomendacao",
    "dtAprovDiretoria",
    "dtAssinatura",
    "dtUltimoDesembolsoOriginal",
    "dtUltimoDesembolsoVigente"
]

# Converter para datetime e substituir datas inválidas por NaT
for col in cols_datas:
    df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
    df[col] = df[col].replace(pd.Timestamp('1900-01-01'), pd.NaT)


# Exibição no formato brasileiro (sem criar novas colunas)
df.head(10).style.format({
    col: lambda t: t.strftime('%d/%m/%Y') if pd.notnull(t) else ''
    for col in cols_datas
})


,cdCartaConsulta,nmProjeto,sgProjeto,nmSetor,nmSujeito,Regiao,nmUnidadeFederal,nmAbrangenciaNacional,TipoFinanciamento,sgFonte,VlDolar,CpVlDolar,sgMoeda,deFase,dtRecebimento,dtValidadeRecomendacao,dtReuniaoNegociacao,dtAprovDiretoria,dtAprovacaoSenado,nuResolucaoSenado,dtAssinatura,nuoperacao,dtPrimeiraAmortizacao,dtUltimaAmortizacao,dtUltimoDesembolsoOriginal,dtUltimoDesembolsoVigente
0,60371,PROJETO ALFA/PROJECT ALFA,nan,nan,Instituto Internacional para Sustentabilidade - IIS,Sudeste,Estado do Rio de Janeiro,Federal,Outros,Ch_Inst,750000000,200000000,US$,Arquivado,03/07/2017,,nan,,nan,nan,,nan,nan,nan,,
1,60371,PROJETO ALFA/PROJECT ALFA,nan,nan,Instituto Internacional para Sustentabilidade - IIS,Sudeste,Estado do Rio de Janeiro,Federal,Outros,Ch_Inst,750000000,250000000,US$,Arquivado,03/07/2017,,nan,,nan,nan,,nan,nan,nan,,
2,12761,Programa de Desenvolvimento do Turismo Nacional - PRODETUR NACIONAL GOIÂNIA,PRODETUR NACIONAL GOIÂNIA,Comércio e Serviços (Turismo),aaa,Centro Oeste,Distrito Federal,Municipal,Operação de Crédito Externo,BID,25000000,25000000,US$,Arquivado,14/08/2008,27/07/2013,nan,,nan,nan,,nan,nan,nan,,
3,59718,Apoio à Execução da Estratégia Nacional de Simplificação do Comércio Exterior,DOAÇÃO PRODEV ABDI,Fortalecimento Institucional,Agência Brasileira de Desenvolvimento Industrial - ABDI,Nacional,Distrito Federal,Federal,Doações,BID/PRODEV,495500,0,US$,Arquivado,04/09/2009,,nan,,nan,nan,,nan,nan,nan,,
4,61246,Programa Novo Seguro de Crédito à Exportação (NSCE) da ABGF,NSCE,Fortalecimento Institucional,Agência Brasileira Gestora de Fundos Garantidores e Garantias S.A.,Nacional,Distrito Federal,Federal,Operação de Crédito Externo,BID,300000000,0,US$,Em preparação,05/11/2024,18/12/2026,nan,,nan,nan,,nan,nan,nan,,
5,58612,Projeto Piloto de Assentamento Privado,ASSENTAMENTO PRIVADO,Fortalecimento Institucional,Agência de Desenvolvimento do Mato Grosso do Sul,Centro Oeste,Estado do Mato Grosso do Sul,Estadual,Doações,BID/FUMIN,690000,690000,US$,Arquivado,02/03/1999,,nan,,nan,nan,,nan,nan,nan,,
6,60890,Programa de Financiamento para a Recuperação Produtiva e Sustentável das MPMEs em Santa Catarina,Financiamento a MPMEs em Santa Catarina,Crédito,Agência de Fomento do Estado de Santa Catarina S/A - BADESC,Sul,Estado de Santa Catarina,Estadual,Operação de Crédito Externo,BID,50000000,0,US$,Aguardando Assinatura,16/08/2022,11/11/2024,16/10/2023 00:00,,nan,nan,,nan,nan,nan,,
7,60672,Programa Desenvolve SP - Infraestruturas Sustentáveis,Desenvolve SP - Infra. Sustentáveis,Crédito,Agência de Fomento do Estado de São Paulo - Desenvolve SP,Sudeste,Estado de São Paulo,Estadual,Operação de Crédito Externo,BID,110000000,0,US$,Aguardando Assinatura,27/09/2021,08/11/2023,16/09/2022 00:00,,nan,nan,,nan,nan,nan,,
8,60672,Programa Desenvolve SP - Infraestruturas Sustentáveis,Desenvolve SP - Infra. Sustentáveis,Crédito,Agência de Fomento do Estado de São Paulo - Desenvolve SP,Sudeste,Estado de São Paulo,Estadual,Operação de Crédito Externo,NDB,90000000,0,US$,Aguardando Assinatura,27/09/2021,06/02/2022,10/12/2020 00:00,22/03/2022,nan,nan,,nan,nan,nan,,
9,60743,Novos instrumentos de financiamento de inovação para o Estado de São Paulo,Inova SP,Crédito,Agência de Fomento do Estado de São Paulo - Desenvolve SP,Sudeste,Estado de São Paulo,Estadual,Operação de Crédito Externo,BID,195000000,0,US$,Aguardando Assinatura,26/11/2020,30/12/2022,22/11/2021 00:00,,nan,nan,,nan,nan,nan,,


# ---------- REMOVER COLUNAS DESNECESSÁRIAS ----------

In [ ]:

colunas_para_dropar = [
    "dtReuniaoNegociacao",
    "dtAprovacaoSenado",
    "dtPrimeiraAmortizacao",
    "dtUltimaAmortizacao"
]
df = df.drop(columns=colunas_para_dropar)

# ---------- FLAGS DE DATAS ----------

In [ ]:
df['dias_ate_assinatura'] = (df['dtAssinatura'] - df['dtRecebimento']).dt.days
df['dias_ate_ultimo_desembolso'] = (df['dtUltimoDesembolsoVigente'] - df['dtRecebimento']).dt.days
df['projeto_assinado'] = df['dtAssinatura'].notnull()
df['projeto_desembolsado'] = df['dtUltimoDesembolsoVigente'].notnull()
df['dias_entre_assinatura_e_desembolso'] = (df['dtUltimoDesembolsoVigente'] - df['dtAssinatura']).dt.days

# ---------- FLAGS DE MULTIFONTE ----------

In [ ]:
df['projeto_multifonte'] = df.duplicated(subset=['cdCartaConsulta'], keep=False)

In [ ]:
df.columns

Index(['cdCartaConsulta', 'nmProjeto', 'sgProjeto', 'nmSetor', 'nmSujeito',
       'Regiao', 'nmUnidadeFederal', 'nmAbrangenciaNacional',
       'TipoFinanciamento', 'sgFonte', 'VlDolar', 'CpVlDolar', 'sgMoeda',
       'deFase', 'dtRecebimento', 'dtValidadeRecomendacao', 'dtAprovDiretoria',
       'nuResolucaoSenado', 'dtAssinatura', 'nuoperacao',
       'dtUltimoDesembolsoOriginal', 'dtUltimoDesembolsoVigente',
       'dias_ate_assinatura', 'dias_ate_ultimo_desembolso', 'projeto_assinado',
       'projeto_desembolsado', 'dias_entre_assinatura_e_desembolso',
       'projeto_multifonte'],
      dtype='object')

In [ ]:
# Converter colunas monetárias com vírgula e notação científica
for col in ['VlDolar', 'CpVlDolar']:
    df[col] = df[col].astype(str).str.replace(',', '.', regex=False)  # vírgula -> ponto
    df[f'{col}_num'] = pd.to_numeric(df[col], errors='coerce')        # converte pra número



In [ ]:
resumo_projeto = df.groupby(['cdCartaConsulta', 'sgFonte']).agg(
    total_VlDolar=('VlDolar_num', 'sum'),
    total_CpVlDolar=('CpVlDolar_num', 'sum'),
    qtd_registros=('nmProjeto', 'count')
).reset_index()


In [ ]:
# Contar quantidade de fontes diferentes
fontes_por_projeto = df.groupby('cdCartaConsulta')['sgFonte'].nunique().reset_index()
fontes_por_projeto.rename(columns={'sgFonte': 'qtd_fontes'}, inplace=True)
fontes_por_projeto['projeto_multi_fonte'] = fontes_por_projeto['qtd_fontes'] > 1


In [ ]:
# Merge com dataframe original
df = df.merge(fontes_por_projeto[['cdCartaConsulta', 'projeto_multi_fonte']], on='cdCartaConsulta', how='left')

In [ ]:
# Renomeia a coluna antiga para deixar o propósito claro
df.rename(columns={'projeto_multifonte': 'projeto_duplicado'}, inplace=True)

# Mostra as duas flags lado a lado pra comparar e validar
df[['cdCartaConsulta', 'sgFonte', 'projeto_duplicado', 'projeto_multi_fonte']].head(10)

,cdCartaConsulta,sgFonte,projeto_duplicado,projeto_multi_fonte
0,60371,Ch_Inst,True,False
1,60371,Ch_Inst,True,False
2,12761,BID,False,False
3,59718,BID/PRODEV,False,False
4,61246,BID,False,False
5,58612,BID/FUMIN,False,False
6,60890,BID,False,False
7,60672,BID,True,True
8,60672,NDB,True,True
9,60743,BID,False,False


In [ ]:
# Agrupar valores em dólar como float
df['VlDolar_num'] = df['VlDolar'].replace('[\$,]', '', regex=True).astype(float)
df['CpVlDolar_num'] = df['CpVlDolar'].replace('[\$,]', '', regex=True).astype(float)

<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:3: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:3: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-1178670134.py:2: SyntaxWarning: invalid escape sequence '\$'
  df['VlDolar_num'] = df['VlDolar'].replace('[\$,]', '', regex=True).astype(float)
/tmp/ipython-input-1178670134.py:3: SyntaxWarning: invalid escape sequence '\$'
  df['CpVlDolar_num'] = df['CpVlDolar'].replace('[\$,]', '', regex=True).astype(float)


In [ ]:
# Total por cdCartaConsulta e fonte
resumo_projeto = df.groupby(['cdCartaConsulta', 'sgFonte']).agg(
    total_VlDolar=('VlDolar_num', 'sum'),
    total_CpVlDolar=('CpVlDolar_num', 'sum'),
    qtd_registros=('nmProjeto', 'count')
).reset_index()

In [ ]:
# ---------- COLUNAS DERIVADAS DE ANO ----------
df['ano_recebimento'] = df['dtRecebimento'].dt.year
df['ano_assinatura'] = df['dtAssinatura'].dt.year


# ---------- PADRONIZAÇÃO DE FASE ----------

In [ ]:
def fase_grupo(fase):
    if fase == 'Arquivado':
        return 'Arquivado'
    elif fase in ['Em execução', 'Repagamento', 'Finalizada']:
        return 'Ativo/Executado'
    elif fase in ['Aguardando Assinatura', 'Aguardando Cofiex', 'Aguardando negociação']:
        return 'Aguardando ação'
    elif fase in ['Em preparação', 'Em negociação', 'Análise GTEC', 'Em revisão',
                 'Confirmado na Pré-Cofiex', 'Aprovada COFIEX', 'Encaminhada']:
        return 'Em tramitação'
    else:
        return 'Outro'

df['fase_tipo'] = df['deFase'].apply(fase_grupo)

# Contagem de projetos por tipo de fase
resumo_fases = df['fase_tipo'].value_counts().reset_index()
resumo_fases.columns = ['fase_tipo', 'quantidade_projetos']

resumo_fases

,fase_tipo,quantidade_projetos
0,Ativo/Executado,1374
1,Arquivado,1309
2,Em tramitação,159
3,Aguardando ação,135


# ---------- DATAFRAME FINAL ----------

In [ ]:
# Escolher apenas colunas que serão úteis para análise/Locker Studio
colunas_final = [
    'cdCartaConsulta', 'nmProjeto', 'nmSetor', 'Regiao', 'nmAbrangenciaNacional',
    'sgFonte', 'VlDolar_num', 'CpVlDolar_num',
    'dtRecebimento', 'dtValidadeRecomendacao', 'dtAprovDiretoria',
    'dtAssinatura', 'dtUltimoDesembolsoOriginal', 'dtUltimoDesembolsoVigente',
    'ano_recebimento', 'ano_assinatura', 'fase_tipo',
    'dias_ate_assinatura', 'dias_ate_ultimo_desembolso', 'dias_entre_assinatura_e_desembolso',
    'projeto_assinado', 'projeto_desembolsado', 'projeto_multi_fonte'
]

df_final = df[colunas_final]

df_final.rename(columns={'VlDolar_num': 'VlDolar', 'CpVlDolar_num': 'CpVlDolar'}, inplace=True)

# Conferir as primeiras linhas
df_final.head(20)

/tmp/ipython-input-9823735.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.rename(columns={'VlDolar_num': 'VlDolar', 'CpVlDolar_num': 'CpVlDolar'}, inplace=True)


,cdCartaConsulta,nmProjeto,nmSetor,Regiao,nmAbrangenciaNacional,sgFonte,VlDolar,CpVlDolar,dtRecebimento,dtValidadeRecomendacao,...,dtUltimoDesembolsoVigente,ano_recebimento,ano_assinatura,fase_tipo,dias_ate_assinatura,dias_ate_ultimo_desembolso,dias_entre_assinatura_e_desembolso,projeto_assinado,projeto_desembolsado,projeto_multi_fonte
0,60371,PROJETO ALFA/PROJECT ALFA,NaN,Sudeste,Federal,Ch_Inst,750000000.00,200000000.00,2017-07-03,NaT,...,NaT,2017,NaN,Arquivado,NaN,NaN,NaN,False,False,False
1,60371,PROJETO ALFA/PROJECT ALFA,NaN,Sudeste,Federal,Ch_Inst,750000000.00,250000000.00,2017-07-03,NaT,...,NaT,2017,NaN,Arquivado,NaN,NaN,NaN,False,False,False
2,12761,Programa de Desenvolvimento do Turismo Naciona...,Comércio e Serviços (Turismo),Centro Oeste,Municipal,BID,25000000.00,25000000.00,2008-08-14,2013-07-27,...,NaT,2008,NaN,Arquivado,NaN,NaN,NaN,False,False,False
3,59718,Apoio à Execução da Estratégia Nacional de Sim...,Fortalecimento Institucional,Nacional,Federal,BID/PRODEV,495500.00,0.00,2009-09-04,NaT,...,NaT,2009,NaN,Arquivado,NaN,NaN,NaN,False,False,False
4,61246,Programa Novo Seguro de Crédito à Exportação (...,Fortalecimento Institucional,Nacional,Federal,BID,300000000.00,0.00,2024-11-05,2026-12-18,...,NaT,2024,NaN,Em tramitação,NaN,NaN,NaN,False,False,False
5,58612,Projeto Piloto de Assentamento Privado,Fortalecimento Institucional,Centro Oeste,Estadual,BID/FUMIN,690000.00,690000.00,1999-03-02,NaT,...,NaT,1999,NaN,Arquivado,NaN,NaN,NaN,False,False,False
6,60890,Programa de Financiamento para a Recuperação P...,Crédito,Sul,Estadual,BID,50000000.00,0.00,2022-08-16,2024-11-11,...,NaT,2022,NaN,Aguardando ação,NaN,NaN,NaN,False,False,False
7,60672,Programa Desenvolve SP - Infraestruturas Suste...,Crédito,Sudeste,Estadual,BID,110000000.00,0.00,2021-09-27,2023-11-08,...,NaT,2021,NaN,Aguardando ação,NaN,NaN,NaN,False,False,True
8,60672,Programa Desenvolve SP - Infraestruturas Suste...,Crédito,Sudeste,Estadual,NDB,90000000.00,0.00,2021-09-27,2022-02-06,...,NaT,2021,NaN,Aguardando ação,NaN,NaN,NaN,False,False,True
9,60743,Novos instrumentos de financiamento de inovaçã...,Crédito,Sudeste,Estadual,BID,195000000.00,0.00,2020-11-26,2022-12-30,...,NaT,2020,NaN,Aguardando ação,NaN,NaN,NaN,False,False,False


In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2977 entries, 0 to 2976
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   cdCartaConsulta                     2977 non-null   int64         
 1   nmProjeto                           2977 non-null   object        
 2   nmSetor                             2891 non-null   object        
 3   Regiao                              2977 non-null   object        
 4   nmAbrangenciaNacional               2977 non-null   object        
 5   sgFonte                             2976 non-null   object        
 6   VlDolar                             2977 non-null   float64       
 7   CpVlDolar                           2977 non-null   float64       
 8   dtRecebimento                       2977 non-null   datetime64[ns]
 9   dtValidadeRecomendacao              879 non-null    datetime64[ns]
 10  dtAprovDiretoria        

In [ ]:
df.head()

,cdCartaConsulta,nmProjeto,sgProjeto,nmSetor,nmSujeito,Regiao,nmUnidadeFederal,nmAbrangenciaNacional,TipoFinanciamento,sgFonte,...,projeto_assinado,projeto_desembolsado,dias_entre_assinatura_e_desembolso,projeto_duplicado,VlDolar_num,CpVlDolar_num,projeto_multi_fonte,ano_recebimento,ano_assinatura,fase_tipo
0,60371,PROJETO ALFA/PROJECT ALFA,NaN,NaN,Instituto Internacional para Sustentabilidade...,Sudeste,Estado do Rio de Janeiro,Federal,Outros,Ch_Inst,...,False,False,NaN,True,750000000.00,200000000.00,False,2017,NaN,Arquivado
1,60371,PROJETO ALFA/PROJECT ALFA,NaN,NaN,Instituto Internacional para Sustentabilidade...,Sudeste,Estado do Rio de Janeiro,Federal,Outros,Ch_Inst,...,False,False,NaN,True,750000000.00,250000000.00,False,2017,NaN,Arquivado
2,12761,Programa de Desenvolvimento do Turismo Naciona...,PRODETUR NACIONAL GOIÂNIA,Comércio e Serviços (Turismo),aaa,Centro Oeste,Distrito Federal,Municipal,Operação de Crédito Externo,BID,...,False,False,NaN,False,25000000.00,25000000.00,False,2008,NaN,Arquivado
3,59718,Apoio à Execução da Estratégia Nacional de Sim...,DOAÇÃO PRODEV ABDI,Fortalecimento Institucional,Agência Brasileira de Desenvolvimento Industri...,Nacional,Distrito Federal,Federal,Doações,BID/PRODEV,...,False,False,NaN,False,495500.00,0.00,False,2009,NaN,Arquivado
4,61246,Programa Novo Seguro de Crédito à Exportação (...,NSCE,Fortalecimento Institucional,Agência Brasileira Gestora de Fundos Garantido...,Nacional,Distrito Federal,Federal,Operação de Crédito Externo,BID,...,False,False,NaN,False,300000000.00,0.00,False,2024,NaN,Em tramitação


# --------------- Exportação final ----------------

In [ ]:
# Garantir que todas as strings estão limpas
for col in df_final.select_dtypes(include='object').columns:
    df_final.loc[:, col] = df_final[col].str.strip()

df_final.to_csv(
    '/content/drive/MyDrive/Projeto EBAC FINAL/dados_final.csv',
    index=False,
    sep=';',       # separador de colunas
    decimal=','    # separador decimal
)